# Portfolio Case: Cleaning & Geocoding Outlet Data
# Tujuan: 
# Membersihkan data outlet yang memiliki Latitude dan Longitude = 0
# Menampilkan peta interaktif
# Tools: Python, Pandas, Geopy, Folium
# ===========================================

In [1]:
# --- Import Library ---
import pandas as pd
from geopy.geocoders import Nominatim
import time
import folium

# ===========================================
# Load Data
# ===========================================#

In [2]:
# ===========================================
# --- Opsi 1: Gunakan Data Dummy ---
# ===========================================
# --- Pilihan Load Data ---
use_dummy = True  # True = pakai data dummy, False = pakai Excel sendiri

if use_dummy:
    data = {
        "CustomerID": [f"C{i:03d}" for i in range(1, 11)],
        "CustomerName": [f"Outlet {i}" for i in range(1, 11)],
        "Address": [
            "Jl. Malioboro No.1, Yogyakarta",
            "Jl. Sosrowijayan No.5, Yogyakarta",
            "Jl. Prawirotaman No.10, Yogyakarta",
            "Jl. Ahmad Dahlan No.12, Yogyakarta",
            "Jl. Gejayan No.20, Yogyakarta",
            "Jl. Monjali No.3, Yogyakarta",
            "Jl. Kaliurang No.15, Yogyakarta",
            "Jl. Magelang No.7, Yogyakarta",
            "Jl. Janti No.9, Yogyakarta",
            "Jl. Parangtritis No.4, Yogyakarta"
        ],
        "Latitude": [0]*10,
        "Longitude": [0]*10
    }
    df = pd.DataFrame(data)
else:
    # ===========================================
    # --- Opsi 2: Gunakan Excel sendiri ---
    # 1. Simpan file Excel di folder yang sama dengan notebook ini
    # 2. Ganti nama file di bawah sesuai file Anda
    # ===========================================
    excel_file = "my_outlet_data.xlsx"  # ganti sesuai file
    df = pd.read_excel(excel_file)
    # --- Preview Data Dummy ---
if use_dummy:
    data = {
        "CustomerID": [f"C{i:03d}" for i in range(1, 11)],
        "CustomerName": [f"Outlet {i}" for i in range(1, 11)],
        "Address": [
            "Jl. Malioboro No.1, Yogyakarta",
            "Jl. Sosrowijayan No.5, Yogyakarta",
            "Jl. Prawirotaman No.10, Yogyakarta",
            "Jl. Ahmad Dahlan No.12, Yogyakarta",
            "Jl. Gejayan No.20, Yogyakarta",
            "Jl. Monjali No.3, Yogyakarta",
            "Jl. Kaliurang No.15, Yogyakarta",
            "Jl. Magelang No.7, Yogyakarta",
            "Jl. Janti No.9, Yogyakarta",
            "Jl. Parangtritis No.4, Yogyakarta"
        ],
        "Latitude": [0]*10,
        "Longitude": [0]*10
    }
    df = pd.DataFrame(data)

# Tampilkan 5 baris pertama
df.head()

,CustomerID,CustomerName,Address,Latitude,Longitude
0,C001,Outlet 1,"Jl. Malioboro No.1, Yogyakarta",0,0
1,C002,Outlet 2,"Jl. Sosrowijayan No.5, Yogyakarta",0,0
2,C003,Outlet 3,"Jl. Prawirotaman No.10, Yogyakarta",0,0
3,C004,Outlet 4,"Jl. Ahmad Dahlan No.12, Yogyakarta",0,0
4,C005,Outlet 5,"Jl. Gejayan No.20, Yogyakarta",0,0


# ===========================================
# Data Cleaning
# ===========================================

In [3]:
df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')
df.loc[df['Latitude'] == 0, 'Latitude'] = None
df.loc[df['Longitude'] == 0, 'Longitude'] = None

df.head()

,CustomerID,CustomerName,Address,Latitude,Longitude
0,C001,Outlet 1,"Jl. Malioboro No.1, Yogyakarta",NaN,NaN
1,C002,Outlet 2,"Jl. Sosrowijayan No.5, Yogyakarta",NaN,NaN
2,C003,Outlet 3,"Jl. Prawirotaman No.10, Yogyakarta",NaN,NaN
3,C004,Outlet 4,"Jl. Ahmad Dahlan No.12, Yogyakarta",NaN,NaN
4,C005,Outlet 5,"Jl. Gejayan No.20, Yogyakarta",NaN,NaN


# ===========================================
# Geocoding
# ===========================================

In [ ]:
geolocator = Nominatim(user_agent="geoapi")

def get_latlon(address):
    try:
        loc = geolocator.geocode(address, timeout=10)
        if loc:
            return loc.latitude, loc.longitude
    except:
        return None, None
    return None, None

for i, row in df.iterrows():
    if pd.isna(row['Latitude']) or pd.isna(row['Longitude']):
        lat, lon = get_latlon(row['Address'])
        df.at[i, 'Latitude'] = lat
        df.at[i, 'Longitude'] = lon
        print(f"Updated {row['CustomerName']} → {lat}, {lon}")
        time.sleep(1)  # delay agar tidak diblokir server OSM


Updated Outlet 1 → None, None
Updated Outlet 2 → None, None
Updated Outlet 3 → None, None
Updated Outlet 4 → None, None


# ===========================================
# Simpan hasil bersih
# ===========================================

In [ ]:
df.to_excel("outlet_data_clean.xlsx", index=False)
print("File outlet_data_clean.xlsx berhasil dibuat!")
#df.to_excel(r"C:\Users\Username\Documents\ProjectJupyter\outlet_data_clean.xlsx", index=False)

File outlet_data_clean.xlsx berhasil dibuat!


# ===========================================
# Visualisasi Peta
# ===========================================

In [ ]:
center_lat = df['Latitude'].mean()
center_lon = df['Longitude'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

for _, row in df.iterrows():
    if pd.notna(row['Latitude']) and pd.notna(row['Longitude']):
        folium.Marker([row['Latitude'], row['Longitude']], popup=row['CustomerName']).add_to(m)

m.save("outlet_map.html")
m